<h2>Import libraries</h2>

In [1]:
import numpy as np
import folium
import pandas as pd
from bs4 import BeautifulSoup
import requestsloc = pd.read_csv('')
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print("Libraries imported.")

Libraries imported.


<h1>Web scrapping and wrangling<h1>

In [2]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(url, 'html.parser')

In [3]:
Postalcode=[]
borough=[]
neighborhood=[]
for i in soup.find('table').find_all('tr'):
        data = i.find_all('td')
        if (len(data)>0):
            Postalcode.append(data[0].text.rstrip('\n'))
            borough.append(data[1].text.rstrip('\n'))
            neighborhood.append(data[2].text.rstrip('\n'))

In [6]:
toronto = pd.DataFrame({"postalcode": Postalcode,"borough": borough,"neighborhood": neighborhood})

In [8]:
toronto.head()

,postalcode,borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
toronto.drop(toronto[toronto['borough']=='Not assigned'].index,inplace=True)

In [22]:
t = toronto.reset_index()

In [24]:
t1= t.drop('index',axis=1)

In [25]:
t1.head()

,postalcode,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [26]:
t1 = t1.groupby(['postalcode','borough'],as_index=False).agg(lambda x:','.join(x))
t1.head()

,postalcode,borough,neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [29]:
for index,row in t1.iterrows():
    if row['neighborhood'] == 'Not assigned':
        row['neighborhood'] = row['borough']

In [31]:
t1.shape

(103, 3)

In [32]:
locations = pd.read_csv('http://cocl.us/Geospatial_data')
locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
locations.rename(columns={"Postal Code": "postalcode"}, inplace=True)

In [41]:
t1=t1.merge(locations,on='postalcode',how='left')

In [42]:
t1.head()

,postalcode,borough,neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [45]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h1>lets visualize using Folium </h1>

In [46]:
toranto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(t1['Latitude'], t1['Longitude'], t1['borough'], t1['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toranto)  
toranto

In [51]:
borough_names = list(t1.borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [53]:
t1 = t1[t1['borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(t1.shape)
t1.head()

(39, 5)


,postalcode,borough,neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [47]:
CLIENT_ID = 'MVMTVWURHBFHRUULF5DZH2S5PIIZMRIHENZRD330VKSEF11G' # your Foursquare ID
CLIENT_SECRET = 'PQP2DV2PCZNC5PEOUHX1ZNS15RJZESONF2HAUFSRKK3GI3MI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MVMTVWURHBFHRUULF5DZH2S5PIIZMRIHENZRD330VKSEF11G
CLIENT_SECRET:PQP2DV2PCZNC5PEOUHX1ZNS15RJZESONF2HAUFSRKK3GI3MI


In [59]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(t1['Latitude'], t1['Longitude'], t1['postalcode'], t1['borough'], t1['neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [60]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()


(1647, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [62]:
venues_df.groupby('Neighborhood').count()

,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,,,
Berczy Park,57,57,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16,16,16
Central Bay Street,67,67,67,67,67,67,67,67
Christie,16,16,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100,100,100
Davisville,36,36,36,36,36,36,36,36


In [65]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 234 uniques categories.


# one hot encoding and clustering

In [66]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1647, 237)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 237)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.024390,...,0.024390,0.024390,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.024390
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.050000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.025000,0.000000,0.00,0.025000
4,M4N,Central Toronto,Lawrence Park,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.043478
7,M4S,Central Toronto,Davisville,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.333333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.0000,0.0000,0.000,0.000,0.000,0.062500,...,0.000000,0.000000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.00,0.000000


In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Trail,Neighborhood,Health Food Store,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Restaurant,Yoga Studio,Liquor Store,Spa,Juice Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Brewery,Movie Theater,Ice Cream Shop,Sushi Restaurant,Fast Food Restaurant,Restaurant,Pub,Italian Restaurant,Board Shop
3,M4M,East Toronto,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,M4N,Central Toronto,Lawrence Park,Dim Sum Restaurant,Park,Bus Line,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,M4P,Central Toronto,Davisville North,Gym,Hotel,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,General Travel,Gluten-free Restaurant,Eastern European Restaurant
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Clothing Store,Coffee Shop,Yoga Studio,Mexican Restaurant,Shoe Store,Seafood Restaurant,Salon / Barbershop,Restaurant,Pet Store,Park
7,M4S,Central Toronto,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Gym,Toy / Game Store,Café,Pharmacy
8,M4T,Central Toronto,"Moore Park, Summerhill East",Restaurant,Park,Trail,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,Bagel Shop,Pizza Place,Supermarket,Sushi Restaurant,Fried Chicken Joint,Sports Bar,Restaurant,Bank


In [69]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 2, 2, 0, 0, 4, 0], dtype=int32)

In [74]:
t1.head()
t1.rename(columns={'postalcode':'PostalCode'},inplace=True)

In [75]:
toronto_merged = t1.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head()

(39, 16)


,PostalCode,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Health Food Store,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Restaurant,Yoga Studio,Liquor Store,Spa,Juice Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Park,Brewery,Movie Theater,Ice Cream Shop,Sushi Restaurant,Fast Food Restaurant,Restaurant,Pub,Italian Restaurant,Board Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Fish Market,Pet Store,Park
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Dim Sum Restaurant,Park,Bus Line,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [76]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,borough,neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Health Food Store,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
20,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Japanese Restaurant,Italian Restaurant,Beer Bar
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,Beer Bar
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Café,Sandwich Place,Coffee Shop,Pizza Place,Pub,Donut Shop,BBQ Joint,Indian Restaurant,History Museum,Pharmacy
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Restaurant,Bar,Bookstore,Sandwich Place,Japanese Restaurant,Bakery,Chinese Restaurant,Dessert Shop,Pub
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Burger Joint,Bakery,Grocery Store,Pizza Place
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Harbor / Marina,Bar,Rental Car Location
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Coffee Shop,Café,Hotel,Pub,Seafood Restaurant,Restaurant,Gym,Beer Bar,Japanese Restaurant,Italian Restaurant
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,American Restaurant,Steakhouse
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Diner,Restaurant,Candy Store,Baby Store,Coffee Shop


# visualizing our clusters

In [78]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['borough'], toronto_merged['neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Trail,Neighborhood,Health Food Store,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
20,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Japanese Restaurant,Italian Restaurant,Beer Bar
21,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,Beer Bar
24,Central Toronto,0,Café,Sandwich Place,Coffee Shop,Pizza Place,Pub,Donut Shop,BBQ Joint,Indian Restaurant,History Museum,Pharmacy
25,Downtown Toronto,0,Café,Restaurant,Bar,Bookstore,Sandwich Place,Japanese Restaurant,Bakery,Chinese Restaurant,Dessert Shop,Pub
26,Downtown Toronto,0,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Burger Joint,Bakery,Grocery Store,Pizza Place
27,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Harbor / Marina,Bar,Rental Car Location
28,Downtown Toronto,0,Coffee Shop,Café,Hotel,Pub,Seafood Restaurant,Restaurant,Gym,Beer Bar,Japanese Restaurant,Italian Restaurant
29,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,American Restaurant,Steakhouse
30,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Diner,Restaurant,Candy Store,Baby Store,Coffee Shop


In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Trail,Neighborhood,Health Food Store,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
20,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Japanese Restaurant,Italian Restaurant,Beer Bar
21,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,Beer Bar
24,Central Toronto,0,Café,Sandwich Place,Coffee Shop,Pizza Place,Pub,Donut Shop,BBQ Joint,Indian Restaurant,History Museum,Pharmacy
25,Downtown Toronto,0,Café,Restaurant,Bar,Bookstore,Sandwich Place,Japanese Restaurant,Bakery,Chinese Restaurant,Dessert Shop,Pub
26,Downtown Toronto,0,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Burger Joint,Bakery,Grocery Store,Pizza Place
27,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Harbor / Marina,Bar,Rental Car Location
28,Downtown Toronto,0,Coffee Shop,Café,Hotel,Pub,Seafood Restaurant,Restaurant,Gym,Beer Bar,Japanese Restaurant,Italian Restaurant
29,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,American Restaurant,Steakhouse
30,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Diner,Restaurant,Candy Store,Baby Store,Coffee Shop


In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,1,Park,Jewelry Store,Trail,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,2,Gym,Hotel,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,General Travel,Gluten-free Restaurant,Eastern European Restaurant
4,Central Toronto,2,Dim Sum Restaurant,Park,Bus Line,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
2,East Toronto,2,Park,Brewery,Movie Theater,Ice Cream Shop,Sushi Restaurant,Fast Food Restaurant,Restaurant,Pub,Italian Restaurant,Board Shop
38,East Toronto,2,Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa


In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Home Service,Music Venue,Garden,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4,Park,Playground,Trail,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
8,Central Toronto,4,Restaurant,Park,Trail,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


<h2>observations</h2>
<p> most of the neighbourhood fall under cluster 0, they have mostly coffee shops and cluster 1 has park, cluster 2 has gym and restraunts with swimming pool has most common , cluster 3 home services, cluster 4 parks and other acticty places </p> 